In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import os
os.chdir("/Users/bytedance/MLTrader-deploy/")
import sys
sys.path.insert(0, ".")
import pandas as pd
from research.orderbook_strategies.utils.factor_analysis import update_signal_results_db
from research.orderbook_strategies.utils.helper import load
import numpy as np
from scipy.stats import spearmanr
import research.orderbook_strategies.utils.helper as helper
import research.orderbook_strategies.utils.factor_analysis as factor_analysis

In [9]:
factor_analysis.update_signal_results_db('signal_result_with_atr')

update all signal results


In [12]:
period_list = [32, 64, 128, 256, 512, 1024, 2048]
dire_signal_list = ["kdj.j", "buy_power", "adx.direc2.1024", "rmi", "volume.advantage", 
                    "kdj.k", "ma.dif.10", "dbook", "range.pos", "nr", 
                    "dbook.spread.diff", "dbook.spread"]
dire_signal_list_period = [signal+".{}".format(period) for period in period_list for signal in dire_signal_list]
range_signal_list = ["std", "trend.index", "atr", "adx.1024"]

dire_signal_with_variance = np.array([])
dire_signal_with_variance_period = []
for dire_signal in dire_signal_list:
    for range_signal in range_signal_list:
        signal_name = dire_signal+"."+range_signal +".2048"
        dire_signal_with_variance = np.append(dire_signal_with_variance, signal_name)
dire_signal_with_variance = list(dire_signal_with_variance)
dire_signal_with_variance_period = [signal+".{}".format(period) for period in period_list for signal in dire_signal_with_variance]
trades_signal = ['trade.buy_power.more_position.12', 'trade.doublebuy.value.ratio', 'trade.buy_power', 'trade.buy_power_count', 'trade.buy_power']
trades_signal_period = [signal+".{}".format(period) for period in np.power(2, range(10, 15)) for signal in trades_signal]

print("#dire = {}, #dire_with_var = {}, #trades = {}".format(len(dire_signal_list_period), len(dire_signal_with_variance_period), len(trades_signal_period)))

#dire = 84, #dire_with_var = 336, #trades = 25


In [13]:
product_list = ["OPUSDT","BTCUSDT", "ETHUSDT", "APTUSDT", "1000PEPEUSDT"]
all_results = pd.read_csv("/Volumes/AdamDrive/experiment_root/orderbook_research/signal_results/signal.csv")

all_signals = all_results['signal_name'].unique()
for signal_name in dire_signal_list_period:
    if signal_name not in all_signals:
        print("{} not exist".format(signal_name))
        
for signal_name in dire_signal_with_variance_period:
    if signal_name not in all_signals:
        print("{} not exist".format(signal_name))
        
for signal_name in trades_signal_period:
    if signal_name not in all_signals:
        print("{} not exist".format(signal_name))

trade.buy_power.more_position.12.1024 not exist
trade.doublebuy.value.ratio.1024 not exist
trade.buy_power.1024 not exist
trade.buy_power_count.1024 not exist
trade.buy_power.1024 not exist
trade.buy_power.more_position.12.2048 not exist
trade.doublebuy.value.ratio.2048 not exist
trade.buy_power.2048 not exist
trade.buy_power_count.2048 not exist
trade.buy_power.2048 not exist
trade.buy_power.more_position.12.4096 not exist
trade.doublebuy.value.ratio.4096 not exist
trade.buy_power.4096 not exist
trade.buy_power_count.4096 not exist
trade.buy_power.4096 not exist
trade.buy_power.more_position.12.8192 not exist
trade.doublebuy.value.ratio.8192 not exist
trade.buy_power.8192 not exist
trade.buy_power_count.8192 not exist
trade.buy_power.8192 not exist
trade.buy_power.more_position.12.16384 not exist
trade.doublebuy.value.ratio.16384 not exist
trade.buy_power.16384 not exist
trade.buy_power_count.16384 not exist
trade.buy_power.16384 not exist


In [3]:

grouper_names = ["grouper_0", "grouper_1", "grouper_2", "grouper_3"]
all_results["grouper_0"] = all_results["signal_name"].astype(str) + "_" + all_results["reverse"].astype(str)
all_results["grouper_1"] = all_results["grouper_0"] + "_" + all_results["product"].astype(str)
all_results["grouper_2"] = all_results["grouper_1"] + "_" + all_results["open"].astype(str)
all_results["grouper_3"] = all_results["grouper_2"] + "_" + all_results["close"].astype(str)

## 先filter 交易次数过少的因子
stats = all_results.pivot_table(index="grouper_0", columns="is_train", aggfunc="median", values='avg.num')
prev = len(stats.index)
valid_str = list(stats[stats[True] > 1].index)
now_have = len(valid_str)
all_results = all_results.query("grouper_0 in @valid_str")
print("keep {}/{}".format(now_have, prev))

keep 563/815


In [4]:
all_results["ultra"] = all_results["profit_factor"] * all_results["win_rate"] * all_results["avg.num"]
all_results.sample(3)

,open_close,open,close,num,avg.num,avg.pnl,total.pnl,sharpe,drawdown,max.drawdown,...,period,date_range,bt_strategy,is_train,bt_stra,grouper_0,grouper_1,grouper_2,grouper_3,ultra
63077,0.01513671875_-0.009082031249999999,0.015137,-0.009082,1890.5,14.655039,-0.000067,-0.126151,-0.501663,0.187851,0.552301,...,32,2023-04-01 to 2023-08-08,signal_result_with_atr,False,signal_result_with_atr,nr.std.2048.32_1,nr.std.2048.32_1_OPUSDT,nr.std.2048.32_1_OPUSDT_0.01513671875,nr.std.2048.32_1_OPUSDT_0.01513671875_-0.00908...,7.537104
171155,0.6054589843749998_-0.36327539062499986,0.605459,-0.363275,323.0,1.334711,-1.974641,-637.808960,-3.289298,594.558838,594.558838,...,256,2022-08-02 to 2023-03-31,signal_result_with_atr,True,signal_result_with_atr,range.pos.256_1,range.pos.256_1_ETHUSDT,range.pos.256_1_ETHUSDT_0.6054589843749998,range.pos.256_1_ETHUSDT_0.6054589843749998_-0....,0.666219
231868,0.55810546875_-0.11162109375000001,0.558105,-0.111621,6736.0,27.834711,-0.022560,-151.961426,-0.543296,312.449097,377.667603,...,64,2022-08-02 to 2023-03-31,signal_result_with_atr,True,signal_result_with_atr,dbook.spread.diff.trend.index.2048.64_-1,dbook.spread.diff.trend.index.2048.64_-1_ETHUSDT,dbook.spread.diff.trend.index.2048.64_-1_ETHUS...,dbook.spread.diff.trend.index.2048.64_-1_ETHUS...,13.692627


## 指标之间的相关性

In [47]:
# metric_list = ["profit_factor", "avg.ret", "sharpe.ret", "win_rate", "ultra", "avg.num"]
# res = spearmanr(all_results[metric_list], axis=0).correlation
# res = pd.DataFrame(res, index=metric_list, columns=metric_list)
# print(res)

In [48]:
# good_strategy = all_results[all_results["is_train"] & (all_results["avg.num"] > 1) & (all_results["sharpe"] > 0.1)]["strategy"].unique()
# good_strategy = list(good_strategy)
# good_results = all_results.loc[np.isin(all_results["strategy"], good_strategy)]

# profit_factor = good_results.pivot_table(index=["strategy"], columns="is_train", aggfunc="mean", values="profit_factor")
# sharpe = good_results.pivot_table(index=["strategy"], columns="is_train", aggfunc="mean", values="sharpe")
# sharpe2 = good_results.pivot_table(index=["strategy"], columns="is_train", aggfunc="mean", values="sharpe.ret")
# avg_pnl = good_results.pivot_table(index=["strategy"], columns="is_train", aggfunc="mean", values="avg.pnl")

# avg_pnl = good_results.pivot_table(index=["strategy"], columns="is_train", aggfunc="mean", values="avg.pnl")
# total_pnl = good_results.pivot_table(index=["strategy"], columns="is_train", aggfunc="mean", values="total.pnl")


# profit_factor_spear = spearmanr(profit_factor[True], profit_factor[False]).correlation
# sharpe_spear = spearmanr(sharpe[True], sharpe[False]).correlation
# sharpe2_spear = spearmanr(sharpe2[True], sharpe2[False]).correlation
# avg_pnl_spear = spearmanr(avg_pnl[True], avg_pnl[False]).correlation
# total_pnl_spear = spearmanr(total_pnl[True], total_pnl[False]).correlation

# profit_factor_spear
# print("good pearson_coff of profit_factor, sharpe1, sharpe2 avg_pnl_spear, total_pnl_spear is: ", profit_factor_spear, sharpe_spear, sharpe2_spear, avg_pnl_spear, total_pnl_spear)

In [ ]:
good_strategy = all_results[all_results["is_train"] & (all_results["avg.num"] > 1) & (all_results["profit_factor"] > 1)]["strategy"].unique()
good_strategy = list(good_strategy)
good_results = all_results.loc[np.isin(all_results["strategy"], good_strategy)]

profit_factor = good_results.pivot_table(index=["strategy"], columns="is_train", aggfunc="mean", values="profit_factor")
sharpe = good_results.pivot_table(index=["strategy"], columns="is_train", aggfunc="mean", values="sharpe")
sharpe2 = good_results.pivot_table(index=["strategy"], columns="is_train", aggfunc="mean", values="sharpe.ret")
avg_pnl = good_results.pivot_table(index=["strategy"], columns="is_train", aggfunc="mean", values="avg.pnl")

avg_pnl = good_results.pivot_table(index=["strategy"], columns="is_train", aggfunc="mean", values="avg.pnl")
total_pnl = good_results.pivot_table(index=["strategy"], columns="is_train", aggfunc="mean", values="total.pnl")


profit_factor_spear = spearmanr(profit_factor[True], profit_factor[False]).correlation
sharpe_spear = spearmanr(sharpe[True], sharpe[False]).correlation
sharpe2_spear = spearmanr(sharpe2[True], sharpe2[False]).correlation
avg_pnl_spear = spearmanr(avg_pnl[True], avg_pnl[False]).correlation
total_pnl_spear = spearmanr(total_pnl[True], total_pnl[False]).correlation

profit_factor_spear
print("good pearson_coff of profit_factor, sharpe1, sharpe2 avg_pnl_spear, total_pnl_spear is: ", profit_factor_spear, sharpe_spear, sharpe2_spear, avg_pnl_spear, total_pnl_spear)

good pearson_coff of profit_factor, sharpe1, sharpe2 avg_pnl_spear, total_pnl_spear is:  0.7008600688728037 0.7845457568356099 0.789767245626018 0.7901004634015509 0.8586211196307656


# 指标的Train和Test一致性分析

- Sharpe 的一致性更高

In [10]:
# from tqdm.notebook import tqdm
# grouper_name = 'grouper_0'
# matrix = ["profit_factor", 'sharpe.ret', 'ultra']
# groupers = all_results[grouper_name].unique()
# stats = all_results.pivot_table(index=grouper_name, columns="is_train", aggfunc="mean", values=matrix)
# mat_dict = {name: np.zeros((2,2 )) for name in matrix}
# for stra in tqdm(groupers):
#     for name in matrix:
#         if "profit_factor" in name:
#             train_good = stats.loc[stra, (name, True)] > 1
#         elif "ultra" in name:
#             train_good = stats.loc[stra, (name, True)] > 1
#         else:
#             train_good = stats.loc[stra, (name, True)] > 0
#         test_good = stats.loc[stra, ('sharpe.ret', False)] > 0
#         if train_good and test_good:
#             mat_dict[name][0][0] += 1
#         if train_good and not test_good:
#             mat_dict[name][0][1] += 1
#         if not train_good and test_good:
#             mat_dict[name][1][0] += 1
#         if not train_good and not test_good:
#             mat_dict[name][1][1] += 1

# def precision(m):
#     return m[0][0] / (m[0][0] + m[0][1])
# def wrap(m):
#     return pd.DataFrame(m, index=["train.good", "train.bad"], columns=["test.good", "test.bad"])
# for name in matrix:
#     print("Confusion Matrix of {}: ".format(name))
#     print(wrap(mat_dict[name]))
#     print(f"Precision = {precision(mat_dict[name]):.2%}")

# from sklearn.metrics import roc_auc_score
# label = stats.loc[:, ('sharpe.ret', False)]
# for name in matrix:
#     pred = stats.loc[:, (name, True)].values
#     pred = (pred - pred.min()) / (pred.max() - pred.min())
#     auc = roc_auc_score(label.values > 0, pred)
#     print(f"AUC of {name} is {auc:.2%}")


"""
    Confusion Matrix of profit_factor: 
            test.good  test.bad
train.good    24337.0   47153.0
train.bad     30939.0   32754.0
Precision = 34.04%
Confusion Matrix of sharpe.ret: 
            test.good  test.bad
train.good    29813.0   25335.0
train.bad     25463.0   54572.0
Precision = 54.06%
Confusion Matrix of ultra: 
            test.good  test.bad
train.good    37016.0   60095.0
train.bad     18260.0   19812.0
Precision = 38.12%

AUC of profit_factor is 40.23%
AUC of sharpe.ret is 66.22%
AUC of ultra is 40.77%
"""
        

### Learderboard

In [42]:
# group_0
group_0_res = all_results.groupby(by=["grouper_3", "is_train"]).mean()
group_0_res_train = group_0_res.loc[(slice(None), True), :].droplevel(1)
group_0_res_test = group_0_res.loc[(slice(None), False), :].droplevel(1)

In [43]:
cols_of_interest = ["sharpe.ret", "avg.num", "avg.ret"]
good_groups_in_train = list(group_0_res_train[group_0_res_train['sharpe.ret'] > 1].index)
print("#good_in_train: {}/{}".format(len(good_groups_in_train), group_0_res_train.shape[0]))

#good_in_train: 26638/135183


In [44]:
test_perf_of_good = group_0_res_test[np.isin(group_0_res_test.index, good_groups_in_train)][cols_of_interest]
test_perf_of_good["sharpe.ret.train"] = test_perf_of_good.index.map(group_0_res_train['sharpe.ret'])

In [45]:
# performance without transaction cost
test_perf_of_good.sort_values(by="sharpe.ret", ascending=False).head(20)

,sharpe.ret,avg.num,avg.ret,sharpe.ret.train
grouper_3,,,,
dbook.spread.diff.atr.2048.32_1_BTCUSDT_0.001906394958496_-0.0003812789916992,13.564837,662.230769,0.000028,13.221459
dbook.spread.diff.atr.2048.32_1_BTCUSDT_0.001906394958496_-0.0007625579833984,13.170792,624.546154,0.000028,12.616034
dbook.spread.diff.adx.1024.2048.32_1_ETHUSDT_0.2030029296875_-0.0406005859375,12.770837,475.953846,0.000032,7.404232
dbook.spread.diff.atr.2048.32_1_ETHUSDT_0.0019855499267578_-0.0003971099853515,12.720686,647.438462,0.000029,6.940251
dbook.spread.diff.adx.1024.2048.32_1_ETHUSDT_0.2030029296875_-0.081201171875,12.577044,445.800000,0.000031,7.390442
dbook.spread.diff.atr.2048.32_1_BTCUSDT_0.0022354125976562_-0.0004470825195312,12.559901,601.669231,0.000027,12.298551
dbook.spread.diff.adx.1024.2048.32_1_BTCUSDT_0.238525390625_-0.09541015625,12.482324,396.115385,0.000029,11.488091
dbook.spread.diff.atr.2048.32_1_BTCUSDT_0.001906394958496_-0.0011438369750976,12.467890,589.653846,0.000028,11.755800
dbook.spread.diff.adx.1024.2048.32_1_BTCUSDT_0.238525390625_-0.047705078125,12.410347,423.600000,0.000028,12.712145


In [46]:
# performance with transaction cost
cols_of_interest = ["sharpe.ret", "avg.num", "avg.ret"]
good_groups_in_train = list(group_0_res_train[(group_0_res_train['sharpe.ret'] > 1) & (group_0_res_train['avg.ret'] > 0.0004)].index)
print("#good_in_train: {}/{}".format(len(good_groups_in_train), group_0_res_train.shape[0]))

test_perf_of_good = group_0_res_test[np.isin(group_0_res_test.index, good_groups_in_train)][cols_of_interest]
test_perf_of_good["sharpe.ret.train"] = test_perf_of_good.index.map(group_0_res_train['sharpe.ret'])

# performance without transaction cost
test_perf_of_good.sort_values(by="sharpe.ret", ascending=False).head(20)


#good_in_train: 14895/135183


,sharpe.ret,avg.num,avg.ret,sharpe.ret.train
grouper_3,,,,
range.pos.std.2048.64_-1_1000PEPEUSDT_0.0194549560546875_-0.0038909912109375,8.165739,8.050000,0.000647,5.289959
kdj.j.trend.index.2048.128_-1_1000PEPEUSDT_0.75_-0.15,8.164916,2.966667,0.001304,1.662785
volume.advantage.std.2048.64_-1_1000PEPEUSDT_0.0193023681640625_-0.0038604736328125,7.125610,5.133333,0.000635,6.231514
dbook.256_1_1000PEPEUSDT_0.1257989501953124_-0.0251597900390624,7.071797,3.533333,0.001249,2.133599
kdj.k.trend.index.2048.256_-1_1000PEPEUSDT_0.7001953125_-0.280078125,6.816256,9.483333,0.000978,1.259304
kdj.j.trend.index.2048.128_-1_1000PEPEUSDT_0.689453125_-0.4136718749999999,6.784769,6.050000,0.001566,1.334482
dbook.32_1_1000PEPEUSDT_0.3426484375000003_-0.2741187500000002,6.651944,17.466667,0.000770,4.343289
range.pos.std.2048.128_-1_1000PEPEUSDT_0.020904541015625_-0.00836181640625,6.528251,2.400000,0.001320,1.421025
rmi.std.2048.32_-1_1000PEPEUSDT_0.0357396850585933_-0.021443811035156,6.424613,8.200000,0.000574,7.583823
